# SRA BioSample data landscape
Adam Klie<br>
10/17/2019<br>
Script to analyze the metadata landscape of BioSample entries in SRA

### Import needed packages and set up plotting

In [1]:
%matplotlib inline
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import brewer2mpl
import seaborn as sns
from IPython.display import display, HTML

In [39]:
from matplotlib import rcParams
import matplotlib as mpl

#colorbrewer2 Dark2 qualitative color table
dark2_colors = brewer2mpl.get_map('Dark2', 'Qualitative', 7).mpl_colors

rcParams['figure.figsize'] = (10, 6)
rcParams['figure.dpi'] = 600
rcParams['axes.prop_cycle'] = mpl.cycler(color=dark2_colors) 
rcParams['lines.linewidth'] = 2
rcParams['axes.facecolor'] = 'white'
rcParams['font.size'] = 14
rcParams['patch.edgecolor'] = 'white'
rcParams['patch.facecolor'] = dark2_colors[0]
rcParams['font.family'] = 'StixGeneral'


def remove_border(axes=None, top=False, right=False, left=True, bottom=True):
    """
    Minimize chartjunk by stripping out unnecesasry plot borders and axis ticks
    
    The top/right/left/bottom keywords toggle whether the corresponding plot border is drawn
    """
    ax = axes or plt.gca()
    ax.spines['top'].set_visible(top)
    ax.spines['right'].set_visible(right)
    ax.spines['left'].set_visible(left)
    ax.spines['bottom'].set_visible(bottom)
    
    #turn off all ticks
    ax.yaxis.set_ticks_position('none')
    ax.xaxis.set_ticks_position('none')
    
    #now re-enable visibles
    if top:
        ax.xaxis.tick_top()
    if bottom:
        ax.xaxis.tick_bottom()
    if left:
        ax.yaxis.tick_left()
    if right:
        ax.yaxis.tick_right()

### Helper Functions to find and plot counts and coverage

In [9]:
def find_coverage(attributes, attributeCounts, num_samples):
    coverage = {}
    for attribute in attributes:
        if attribute in attributeCounts.index:
            coverage[attribute] = attributeCounts[attribute]/num_samples*100
    return(pd.Series(coverage).sort_values(ascending=False))

def find_counts(attributes, attributeCounts, num_samples):
    counts = {}
    for attribute in attributes:
        if attribute in attributeCounts.index:
            counts[attribute] = attributeCounts[attribute]
    return(pd.Series(counts).sort_values(ascending=False))

def plot_attributes(data, n=20, ylab='of samples with attribute', plot_type='%', main='', save=None):
    plt.tight_layout()
    ax = data.iloc[:n].plot.bar()
    ax.set_ylabel(plot_type + ' ' + ylab)
    ax.set_xlabel('Attributes')
    ax.set_title(main)
    for xtick_label in ax.get_xticklabels():
        if xtick_label.get_text() in NCBI_attribute_df.index:
            xtick_label.set_color('red')
        else:
            xtick_label.set_color('blue')
    if save != None:
        plt.savefig(save, dpi=1200, bbox_inches="tight")
            
def plot_mulitple_attributes(data, ax=None, n=10, ylab ='% of samples with attribute', **kwargs):
    ax = ax or plt.gca()
    ax.set_ylabel(ylab)
    ax.set_xlabel('Attributes')
    for xtick_label in ax.get_xticklabels():
        xtick_label.set_rotation(90)
        if xtick_label.get_text() in NCBI_attribute_df.index:
            xtick_label.set_color('red')
        else:
            xtick_label.set_color('blue')
    return ax.bar(data.iloc[:n].index, data.iloc[:n].values)

### 1. Read in downloaded SRA BioSample key-value pairs and NCBI defined attributes

In [10]:
# Load SRA BioSample key-value pairs
SRS_dir = "../data/allSRS_05_15_2018.pickle"
allSRS = pd.read_pickle(SRS_dir)

In [11]:
# Also read in the attributes defined by the NCBI BioSample
NCBI_attribute_df = pd.read_pickle('../data/BioSampleAttributes.pickle')

In [12]:
display(allSRS.head())
NCBI_attribute_df.head()

ERS069057  TITLE                                Streptococcus pneumoniae
           SCIENTIFIC_NAME                      Streptococcus pneumoniae
           SUBMITTER_ID          SN20648-sc-2011-11-03T19:59:41Z-1101437
           Strain                                                       
           Sample Description                                           
dtype: object

,Description,Format,HarmonizedName,Package,Synonym
Name,,,,,
EDTA inhibitor tested,Was carbapenemase activity tested in the prese...,"['', 'yes', 'no', 'missing', 'not applicable',...",edta_inhibitor_tested,Beta-lactamase.1.0,edta inhibitor tested
FAO classification,soil classification from the FAO World Referen...,{term},fao_class,MIUVIG.soil.5.0,soil taxonomic/fao classification
HIV status,"HIV status of subject, if yes HAART initiation...",{boolean};{boolean},hiv_stat,NaN,hiv status
NARMS isolate number,isolate identifier for the collection of isola...,{text},narms_isolate_number,NaN,narms isolate number
Omics Observatory ID,A unique identifier of the omics-enabled obser...,{term},omics_observ_id,MIUVIG.water.5.0,omics_observ_id


#### (Optional) Subset the dataframe for human samples

In [ ]:
scientific_name_m = allSRS.index.get_level_values(1) == 'SCIENTIFIC_NAME'
hs_m = allSRS.values == 'Homo sapiens'
hs = allSRS[scientific_name_m & hs_m].index.get_level_values(0)
hs_SRS =  allSRS[allSRS.index.get_level_values(0).isin(hs)]

#### (Optional) Subset the series for those samples that have a title

In [ ]:
title_m= allSRS[(allSRS.index.get_level_values(1) == 'TITLE')].index.get_level_values(0)
title_SRS = allSRS[allSRS.index.get_level_values(0).isin(title_m)]

#### Choose what data you want to analyze, the entire dataset or some subset, build a dataframe to aid with analysis

In [13]:
currSRS = allSRS  # all of BioSample annotations

In [14]:
# build a dataframe for the key (attribute) and values for each sample (srs)
SRS_df = pd.DataFrame(currSRS).reset_index()
SRS_df.columns = ['srs', 'attribute', 'value']
SRS_df = SRS_df.set_index('srs')

### <font color = red>Figure 1A</font>

In [63]:
classes = ['SCIENTIFIC_NAME', 'genotype', 'strain' 'cell type', 'disease', 'sex', 'age', 'molecular data type', 'platform', 'protocol']

In [72]:
class_SRS_df = SRS_df[SRS_df['attribute'].isin(classes)]

In [73]:
example_df = class_SRS_df.sample(class_SRS_df.shape[0]).groupby('attribute').head(1)
example_df

,attribute,value,attribute_len,log_attribute_len,word_count,log_word_count
srs,,,,,,
SRS441497,platform,Illumina HiSeq 2000,19,4.321928,3,2.000000
SRS438070,SCIENTIFIC_NAME,Homo sapiens,12,3.700440,2,1.584963
SRS1884219,sex,not applicable,14,3.906891,2,1.584963
SRS2222927,molecular data type,SNP Genotypes (NGS),19,4.321928,3,2.000000
SRS1630076,age,2 years,7,3.000000,2,1.584963
SRS752582,disease,Acute Myeloid Leukemia,22,4.523562,3,2.000000
SRS877909,genotype,124,3,2.000000,1,1.000000
SRS1274093,protocol,48h dox induction,17,4.169925,3,2.000000


In [74]:
example_df.to_csv('../results/figures/Figure1A.csv')

### 2. Overall data landscape 

#### Find the total number of annotations and of samples with annotations

In [16]:
attributeCounts = SRS_df['attribute'].value_counts()

In [17]:
outtext = ""

In [18]:
totalAttributes = len(currSRS)
outtext = outtext + ("Number of BioSample key-value pairs: %d\n" % totalAttributes)
totalAttributes

43907007

In [19]:
num_samples = SRS_df.loc[~SRS_df.index.duplicated(keep='first')].shape[0]
outtext = outtext + ("Number of BioSample entries: %d\n" % num_samples)

In [20]:
averageAnnotations = totalAttributes/num_samples
outtext = outtext + ("The average number of attributes per BioSample in SRA: %.2f\n" % averageAnnotations)

In [21]:
print(outtext)

Number of BioSample key-value pairs: 43907007
Number of BioSample entries: 2921722
The average number of attributes per BioSample in SRA: 15.03



### 3. Compare attributes defined by NCBI to the user-defined

In [22]:
m = SRS_df["attribute"].isin(NCBI_attribute_df.index)
in_NCBI = SRS_df[m]
not_in_NCBI = SRS_df[~m]
n = len(in_NCBI.index.unique())
n1 = in_NCBI.shape[0]

In [23]:
outtext = ""

In [24]:
outtext = outtext + ("Number of unique attributes in SRA BioProject Metadata: %d\n" % len((SRS_df['attribute'].unique())))

In [25]:
outtext = outtext + ("Number of attributes defined by BioProject: %d\n" % NCBI_attribute_df.shape[0])

In [26]:
outtext = outtext + ("Number of attributes defined by BioProject actually used: %d\n" % len(in_NCBI['attribute'].unique()))

In [27]:
outtext = outtext + ("Number of attributes used not defined by BioProject: %d\n" % len(not_in_NCBI['attribute'].unique()))

In [28]:
outtext = outtext + ("Number of samples that actually use a BioProject attributes %d, " \
     "which is %.2f%%\n" % (n, 100*n/num_samples))
outtext = outtext + ("Number of keys that actually are BioProject attributes %d, " \
     "which is %.2f%%\n" % (n1, 100*n1/totalAttributes))

In [29]:
print(outtext)

Number of unique attributes in SRA BioProject Metadata: 19361
Number of attributes defined by BioProject: 456
Number of attributes defined by BioProject actually used: 316
Number of attributes used not defined by BioProject: 19045
Number of samples that actually use a BioProject attributes 2183317, which is 74.73%
Number of keys that actually are BioProject attributes 9574210, which is 21.81%



### 4. Determine the coverage of different attributes

In [30]:
# Some samples have multiple values for one key, only keep the first
currSRS = currSRS.loc[~currSRS.index.duplicated(keep='first')] 

# Build a new dataframe for the key (attribute) and values for each sample (srs) nd = no duplicates
SRS_nd = pd.DataFrame(currSRS).reset_index()
SRS_nd.columns = ['srs', 'attribute', 'value']
SRS_nd = SRS_nd.set_index('srs')

In [31]:
attributeCounts = SRS_nd['attribute'].value_counts()

##### Won't normailze at all for sake of exploring issue with coverage of BioSample defined attributes 

In [ ]:
# From this example we also note the normalization problem in this data
#outtext = outtext + ('TITLE: %d\ntitle: %d\nTitle: %d' % 
      #(attributeCounts['TITLE'], attributeCounts['title'], attributeCounts['Title']))

In [ ]:
# Use a normalized (lowercased) version of df for coverage investigation
#SRSlc_df = SRS_nd.apply(lambda x: x.astype(str).str.lower())
#attributeCounts = SRSlc_df['attribute'].value_counts()

In [ ]:
# See that it merges all the titles
#attributeCounts['title']

### <font color=red>Figure 1B</font>

In [32]:
top_attributes = attributeCounts.index
top_attribute_coverage = find_coverage(top_attributes, attributeCounts, num_samples)
#plot_attributes(find_coverage(top_attributes, attributeCounts, num_samples), main="Highest count attributes")

In [75]:
plt.tight_layout()
ax = top_attribute_coverage.iloc[:20].plot.bar()
ax.set_ylabel('% of samples with attribute')
ax.set_xlabel('Attributes')
for xtick_label in ax.get_xticklabels():
    if xtick_label.get_text() in NCBI_attribute_df.index:
        xtick_label.set_color('red')
    else:
        xtick_label.set_color('blue')
plt.savefig('../results/figures/Figure1B.eps', dpi=600, bbox_inches="tight")
plt.savefig('../results/figures/Figure1B.png', dpi=600, bbox_inches="tight")
plt.close();

In [51]:
ncbi_attributes = NCBI_attribute_df.index
#plot_attributes(find_coverage(ncbi_attributes, attributeCounts, num_samples), main="Highest count NCBI defined attributes")

### <font color=red>Figure 1C</font>

In [52]:
important_attributes = ['SCIENTIFIC_NAME', 'strain', 'cell type', 'genotype', 'disease', 'sex', 'age',
                        'molecular data type', 'platform', 'protocol']
important_attribute_coverage = find_coverage(important_attributes, attributeCounts, num_samples)
#plot_attributes(find_coverage(important_attributes, attributeCounts, num_samples), main="Attributes used for classification", save='../results/Figure_1B.eps')

In [76]:
plt.tight_layout()
ax = important_attribute_coverage.iloc[:20].plot.bar()
ax.set_ylabel('% of samples with attribute')
ax.set_xlabel('Attributes')
for xtick_label in ax.get_xticklabels():
    if xtick_label.get_text() in NCBI_attribute_df.index:
        xtick_label.set_color('red')
    else:
        xtick_label.set_color('blue')
plt.savefig('../results/figures/Figure1C.eps', dpi=600, bbox_inches="tight")
plt.savefig('../results/figures/Figure1C.png', dpi=600, bbox_inches="tight")
plt.close();

#### Look at attribute word length and total characters to find possible longer text to pull things out of 

In [54]:
SRS_df['attribute_len'] = SRS_df['value'].str.len()
SRS_df['log_attribute_len'] = np.log2(SRS_df['attribute_len']+1)
SRS_df['word_count'] = (SRS_df['value'].str.count(' ')+1)
SRS_df['log_word_count'] = np.log2(SRS_df['word_count']+1)

In [55]:
recurring_attributeCounts = attributeCounts[attributeCounts > 10000]
SRS_rec = SRS_df[SRS_df.attribute.isin(recurring_attributeCounts.index)]

In [56]:
longest_attributes = SRS_rec.groupby(['attribute']).mean().sort_values(
    'log_attribute_len', ascending = False).iloc[:20].index

In [57]:
dataframe_sub = SRS_df[SRS_df.attribute.isin(longest_attributes)]

### <font color = red>Figure 1D</font>

In [77]:
ax = sns.boxplot(data=SRS_df, x='attribute', y='log_attribute_len', order=longest_attributes)
plt.xticks(rotation=90)
for xtick_label in ax.get_xticklabels():
    if xtick_label.get_text() in NCBI_attribute_df.index:
        xtick_label.set_color('red')
    else:
        xtick_label.set_color('blue')
ax.set_ylabel('Log(average characters in attribute + 1)')
ax.set_xlabel('Attributes')
plt.savefig('../results/figures/Figure1D.eps', dpi=600, bbox_inches="tight")
plt.savefig('../results/figures/Figure1D.png', dpi=600, bbox_inches="tight")
plt.close();

In [ ]:
most_word_attributes = SRS_rec.groupby(['attribute']).mean().sort_values(
    'word_count', ascending = False).iloc[:20].index

In [ ]:
dataframe_sub = SRS_df[SRS_df.attribute.isin(most_word_attributes)]
ax = sns.boxplot(data=SRS_df, x='attribute', y='log_word_count', order=most_word_attributes)
plt.xticks(rotation=90)
for xtick_label in ax.get_xticklabels():
    if xtick_label.get_text() in NCBI_attribute_df.index:
        xtick_label.set_color('red')
    else:
        xtick_label.set_color('blue')
ax.set_ylabel('Log(average words in attribute + 1)')
ax.set_xlabel('Attributes')


In [ ]:
long_character_fraction = len(SRS_df[SRS_df['attribute_len'].values > 50].index.unique())/num_samples*100
outtext = outtext + ("Number of samples with an attribute of 50 characters or more: %.2f%%\n" % 
      long_character_fraction)

In [ ]:
long_word_fraction = len(SRS_df[SRS_df['word_count'].values >= 10].index.unique())/num_samples*100
outtext = outtext + ("Number of samples with an attribute of 10 words or more: %.2f%%\n" % 
      long_word_fraction)

In [ ]:
print(outtext)

#### Analyze high frequency tokens

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from collections import Counter

In [ ]:
import spacy
# Load the installed model "en_core_web_sm"
nlp = spacy.load("en_core_web_sm")

In [ ]:
nlp = spacy.load("en_core_web_sm")
SRA_docs = [nlp(x) for x in SRS_df.value.values[1:100] if len(x) > 0 ]

In [ ]:
tokenizer = TreebankWordTokenizer()
SRA_dict = [tokenizer.tokenize(x.lower()) for x in SRS_df.value.values[1:100]]

In [ ]:
currSRS = allSRS
# build a dataframe for the key (attribute) and values for each sample (srs)
SRS_df = pd.DataFrame(currSRS).reset_index()
SRS_df.columns = ['srs', 'attribute', 'value']
SRS_df = SRS_df.set_index('srs')

In [ ]:
SRS_df['value'].value_counts()

In [ ]:
sum(SRS_df['word_count'].value_counts().iloc[0:3])/SRS_df.shape[0]

In [ ]:
SRS_df['word_count'].value_counts().iloc[5:8]/SRS_df.shape[0]

In [ ]:
SRS_df['attribute'] = "Sex"